<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/viewers_deployment/slim_Firebase_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Slim Deployment with Firebase and Google Cloud

This notebook complements the tutorial available in this document:
[Google Doc: Slim Deployment with Firebase and Google Cloud](https://tinyurl.com/idc-slim-gcp).
You need to complete the steps discussed in that document before proceeding with this notebook! Please use the document above for providing feedback or asking questions.



# Prerequisites

> **User interaction required**: In the following cell, set the variable gcpProjectID to your **Project ID**, gcpClientID to your **Client ID** and FirebaseProjectName to the **name of your Firebase project**.

In [ ]:
gcpProjectID = 'your_project_id'
gcpClientID = '"your_client_id"'
FirebaseProjectName = 'your_firebase_project_name'

## Install yarn

In [ ]:
!npm install -g yarn

## Install Firebase command line tools

In [ ]:
!curl -sL https://firebase.tools | bash

## Clone Slim source code

In [ ]:
!git clone https://github.com/ImagingDataCommons/slim.git

## Install the node version management library (n) to update the node version to match Slim's

In [ ]:
!npm install -g n

## Install and use node 14 using n



In [ ]:
!n install 14 && n 14

# Configure and deploy Slim using Firebase


## Install Slim's dependencies using yarn



In [ ]:
!cd slim && yarn

## Update Slim's configuration file
> The following code snipped configures things like OpenID Connect (OIDC) and server settings.



In [ ]:
settings = """
window.config = {
  // This must match the location configured for web server
  path: '/',
  servers: [
    {
      id: 'example',
      // This must match the proxy location configured for the DICOMweb server
      url:"https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb",
      write: false
    }
  ],
 oidc: {
    authority: "https://accounts.google.com",
    clientId: %s,
    scope: "email profile openid https://www.googleapis.com/auth/cloud-healthcare",
    grantType: "implicit"
  },
  disableWorklist: false,
  disableAnnotationTools: false,
  enableServerSelection: true,
  mode: 'light',
  preload: true,
  annotations: [
    {
      finding: {
        value: '85756007',
        schemeDesignator: 'SCT',
        meaning: 'Tissue'
      },
      findingCategory: {
        value: '91723000',
        schemeDesignator: 'SCT',
        meaning: 'Anatomical structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 255, 0, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      },
    },
    {
      finding: {
        value: '108369006',
        schemeDesignator: 'SCT',
        meaning: 'Tumor'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 0, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '34823008',
        schemeDesignator: 'SCT',
        meaning: 'Tumor necrosis'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 51, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '369705002',
        schemeDesignator: 'SCT',
        meaning: 'Invasive tumor border'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['line', 'freehandline'],
      style: {
        stroke: {
          color: [51, 102, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '399721002',
        schemeDesignator: 'SCT',
        meaning: 'Tumor infiltration by lymphocytes present'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 204, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '47973001',
        schemeDesignator: 'SCT',
        meaning: 'Artifact'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 80, 80, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    }
  ]
}
"""%(gcpClientID)

with open("slim/public/config/example.js", "w") as f:
    f.write(settings)

## Update SLIM's Firebase config to deploy configuration



In [ ]:
settings_firebase = """
{
  "hosting": {
    "public": "build",
    "ignore": [
      "firebase.json",
      "**/.*",
      "**/node_modules/**"
    ],
    "rewrites": [
      {
        "source": "**",
        "destination": "/index.html"
      }
    ]
  }
}
"""

with open("slim/firebase.json", "w") as f:
  f.write(settings_firebase)

## Build SLIM



In [ ]:
!cd slim && REACT_APP_CONFIG=example PUBLIC_URL='/' yarn build

## Login and initialize Firebase configuration

  

> **User interaction required**: In the next step we login to Firebase and configure the setup of Firebase on the VM. When prompted regarding error reporting information, double click next to the prompt, and type your selection. Next, follow the authorization URL to grant the necessary permissions to Firebase, then double-click next to the "Paste authorization code", paste the code and hit "Enter" to complete the process.

In [ ]:
!firebase login --no-localhost

> **User interaction required**: Here you need to login to your Google account where you will be prompted to follow an authorization URL to grant the necessary permissions. Afterwards, we set the default GCP project ID to the one you chose at the beginning of this notebook.   

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project $gcpProjectID
!gcloud config get project

> **User interaction required**: You should have created a Firebase project as a prerequisite step. Select a project or modify SLIM's .firebaserc file which sets the default firebase project.

In [ ]:
!firebase projects:list

In [ ]:
!cd slim && firebase use $FirebaseProjectName

## Deploy
> Finally, we deploy the app. Do not close the notebook, keep the tab open and return to the Google documentation.

In [ ]:
!cd slim && firebase deploy

## Support

If you have any questions about this notebook, please post them in the [IDC User Forum](https://discourse.canceridc.dev).

## Acknowledgments

Imaging Data Commons has been funded in whole or in part with Federal funds from the National Cancer Institute, National Institutes of Health, under Task Order No. HHSN26110071 under Contract No. HHSN261201500003l.

If you use IDC in your research, please cite the following publication:

> Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S. D., Gibbs, D. L., Bridge, C., Herrmann, M. D., Homeyer, A., Lewis, R., Aerts, H. J. W., Krishnaswamy, D., Thiriveedhi, V. K., Ciausu, C., Schacherer, D. P., Bontempi, D., Pihl, T., Wagner, U., Farahani, K., Kim, E. & Kikinis, R. _National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence_. RadioGraphics (2023). [https://doi.org/10.1148/rg.230180](https://doi.org/10.1148/rg.230180)